# Read ASC files

## Import libraries

In [1]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio

## Read DENUE file

In [22]:
gdf = gpd.read_file('data/denue/denue_inegi_final.gpkg')
gdf.drop(columns=['cve_loc',"numero_ext","fecha_alta"], inplace=True)
gdf.head()

,localidad,ageb,manzana,nom_estab,codigo_act,nombre_act,geometry
0,Colima,0727,031,FARMACIA INTERCAMBIABLE,464111,Farmacias sin minisúper,POINT (633121.957 2128794.720)
1,Colima,0746,034,FARMACIA GUADALUPANA,464112,Farmacias con minisúper,POINT (633791.822 2128170.288)
2,Colima,0712,006,FARMACIA NUEVA IMPERIAL,464111,Farmacias sin minisúper,POINT (634823.711 2128769.626)
3,Colima,0750,003,FARMACIA MI BARRIO,464111,Farmacias sin minisúper,POINT (633641.784 2128038.406)
4,Colima,0750,010,FARMACIA LA PROVIDENCIA,464111,Farmacias sin minisúper,POINT (633457.022 2127859.270)


## Ubicación de los archivos ASC

In [3]:
asc_files = glob.glob('data/Rasters/*/*.asc')
asc_files[:5]

['data/Rasters\\Hydraulic\\Depth____0.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10200.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10500.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10800.0.asc',
 'data/Rasters\\Hydraulic\\Depth____11100.0.asc']

# Read SINGLE ASC file

In [34]:
variable = asc_files[0].split('\\')[2].split('_')[0]
variable

'Depth'

In [36]:
step = asc_files[0].split('\\')[2].split('_')[-1][:-4]
step

'0.0'

In [8]:
grid = np.loadtxt(asc_files[0], skiprows=6)
null_value = -9999.
grid[grid == null_value] = np.nan
grid.shape

(1923, 1540)

In [37]:
def read_meta(file_src):
  meta = {}
  with open(file_src, 'r') as f:
    for i in range(6):
      line = f.readline()
      
      # remove empty spaces to keep only one
      line = ' '.join(line.split())
      
      # split line into key and value
      key, value = line.split(' ')
      
      # add key and value to dictionary
      meta[key] = value
  return meta

# Read MULTIPLE ASC files

In [38]:
def read_asc(file_src):
  variable = file_src.split('\\')[2].split('_')[0]
  step = file_src.split('\\')[2].split('_')[-1][:-4]
  
  meta = read_meta(file_src)
  
  grid = np.loadtxt(file_src, skiprows=6)
  null_value = -9999.
  grid[grid == null_value] = np.nan
  
  return grid, meta, variable, step
  
  
read_asc(asc_files[0])

(array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 {'NCOLS': '1540',
  'NROWS': '1923',
  'XLLCENTER': '631153.500000',
  'YLLCENTER': '2125184.250000',
  'CELLSIZE': '5.000',
  'NODATA_VALUE': '-9999.0'},
 'Depth',
 '0.0')

In [39]:
first_grid, first_meta, first_variable, first_step = read_asc(asc_files[0])


In [40]:
first_grid

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [41]:
gdf.head()

,localidad,ageb,manzana,nom_estab,codigo_act,nombre_act,geometry
0,Colima,0727,031,FARMACIA INTERCAMBIABLE,464111,Farmacias sin minisúper,POINT (633121.957 2128794.720)
1,Colima,0746,034,FARMACIA GUADALUPANA,464112,Farmacias con minisúper,POINT (633791.822 2128170.288)
2,Colima,0712,006,FARMACIA NUEVA IMPERIAL,464111,Farmacias sin minisúper,POINT (634823.711 2128769.626)
3,Colima,0750,003,FARMACIA MI BARRIO,464111,Farmacias sin minisúper,POINT (633641.784 2128038.406)
4,Colima,0750,010,FARMACIA LA PROVIDENCIA,464111,Farmacias sin minisúper,POINT (633457.022 2127859.270)


In [57]:
(633121.957 - 631153.5)/5

393.6914000000106

In [110]:
def work_progress(file_src, gdf, df_final):
  grid, meta, variable, step = read_asc(file_src)
  
  for i, row in gdf.iterrows():
    
    x_asc = int((row.geometry.centroid.x - float(meta['XLLCENTER']))/float(meta['CELLSIZE']))
    y_asc = int((row.geometry.centroid.y - float(meta['YLLCENTER']))/float(meta['CELLSIZE']))
    
    v_asc = grid[y_asc, x_asc]
    
    # nom_estab, nombre_act, x_asc, y_asc, v_asc, variable, step
    row_df = [[row.nom_estab, row.nombre_act, x_asc, y_asc, v_asc, variable, step]]
    row_df = pd.DataFrame(row_df, columns=['nom_estab', 'nombre_act', 'x_asc', 'y_asc', 'v_asc', 'variable', 'step'])
    
    df_final = pd.concat([df_final, row_df], ignore_index=True)

  return df_final
  
f = work_progress(asc_files[0], gdf, None)
f

,nom_estab,nombre_act,x_asc,y_asc,v_asc,variable,step
0,FARMACIA INTERCAMBIABLE,Farmacias sin minisúper,393,722,NaN,Depth,0.0
1,FARMACIA GUADALUPANA,Farmacias con minisúper,527,597,NaN,Depth,0.0
2,FARMACIA NUEVA IMPERIAL,Farmacias sin minisúper,734,717,NaN,Depth,0.0
3,FARMACIA MI BARRIO,Farmacias sin minisúper,497,570,NaN,Depth,0.0
4,FARMACIA LA PROVIDENCIA,Farmacias sin minisúper,460,535,NaN,Depth,0.0
...,...,...,...,...,...,...,...
180,UNIVERSIDAD MULTITECNICA PROFESIONAL,Escuelas de educación superior del sector privado,334,642,NaN,Depth,0.0
181,UNIVERSIDAD INETEP INSTITUTO DE EDUCACION TECN...,Escuelas de educación superior del sector privado,428,772,NaN,Depth,0.0
182,USAER ITINERANTE,Escuelas del sector público de educación para ...,682,880,NaN,Depth,0.0
183,VILANOVA COLEGIO 06PJN0049Y PREESCOLAR,Escuelas del sector público que combinan diver...,612,687,NaN,Depth,0.0


In [111]:
df_full = pd.DataFrame(columns=['nom_estab', 'nombre_act', 'x_asc', 'y_asc', 'v_asc', 'variable', 'step'])
for f in asc_files[:2]:
  df_full = work_progress(f, gdf, df_full)

In [112]:
df_full

,nom_estab,nombre_act,x_asc,y_asc,v_asc,variable,step
0,FARMACIA INTERCAMBIABLE,Farmacias sin minisúper,393,722,NaN,Depth,0.0
1,FARMACIA GUADALUPANA,Farmacias con minisúper,527,597,NaN,Depth,0.0
2,FARMACIA NUEVA IMPERIAL,Farmacias sin minisúper,734,717,NaN,Depth,0.0
3,FARMACIA MI BARRIO,Farmacias sin minisúper,497,570,NaN,Depth,0.0
4,FARMACIA LA PROVIDENCIA,Farmacias sin minisúper,460,535,NaN,Depth,0.0
...,...,...,...,...,...,...,...
365,UNIVERSIDAD MULTITECNICA PROFESIONAL,Escuelas de educación superior del sector privado,334,642,NaN,Depth,10200.0
366,UNIVERSIDAD INETEP INSTITUTO DE EDUCACION TECN...,Escuelas de educación superior del sector privado,428,772,1.053163,Depth,10200.0
367,USAER ITINERANTE,Escuelas del sector público de educación para ...,682,880,NaN,Depth,10200.0
368,VILANOVA COLEGIO 06PJN0049Y PREESCOLAR,Escuelas del sector público que combinan diver...,612,687,NaN,Depth,10200.0
